# Web scrapping cities data 

## creating a generic read from the url and target the area to be parsed

In [1]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen
import re
urlbase = 'https://en.wikipedia.org'
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'


def readUrl(url):
    html = urlopen(url)
    bsObj = bsoup(html.read(),'html.parser')
    return bsObj

def infoBoxScraper(link):
    bsObj = readUrl(link)
    target_table = bsObj.find_all('table',{"class":"infobox"})[0]
    return target_table


Here we are calling all the main url and using beautifull soup library to detect the target table 

In [2]:
main_page = readUrl(url)
# locating the target page into html formate
table = main_page.find_all('table',{"class":"wikitable sortable"})[0]
# locating target table which contains all cities name

rows = table.find_all('tr')


Now the we understand the data and each cells how data is stored and formulate our techniques to extract the necessary information. There is always a pattern in an the same columns

In [3]:
data={'city':[], 'state':[], 'population_2018':[], 'population_2010':[], 'change':[],
      'area_in_miles':[], 'area_in_kms':[], 'population_density_in_miles':[], 'population_density_in_kms':[],
     'location':[],'wiki_url':[],'mayor':[],'website':[]}
header = (list) (data.keys())
for row in rows[1:]:
    cells = row.find_all('td')
    
    if cells[1].i and cells[1].i.a:
        wikiurl = urlbase+cells[1].i.a['href']
    else :
        wikiurl = urlbase+cells[1].a['href']
    data['wiki_url'].append(wikiurl)
    
    for i in range(1,len(cells)):
        cell = cells[i]
        if i == 1:
            while str(cell)[0] == '<':
                cell = cell.contents[0]
            data[header[i-1]].append(cell)
            print(cell,end=", ")
        elif i == 7 or i==9:
            nums = re.findall('[\.\d]+',re.sub(',','',cell.contents[0]))
            if len(nums)>0:
                cell = float(nums[0])
            else:
                cell = None
            data[header[i-1]].append(cell)
#             print(cell,end=", ")
        elif i == 10:
            res = ''
            res +=cell.contents[0].contents[0].contents[0].contents[0].contents[0].contents[0].contents[0]+' '
            res += cell.contents[0].contents[0].contents[0].contents[0].contents[0].contents[2].contents[0]
            data[header[i-1]].append(res)
#             print(res)
        else:
            while str(cell)[0] == '<':

                if cell.contents[-1] == '\n' :
                    cell = cell.contents[-2]
                else:
                    cell = cell.contents[-1]
            cell = cell.strip('\n')
            
            if i in [3,4,5,6,8]:
                nums = re.findall('[\.\d]+',re.sub(',','',cell))
                if len(nums)>0:
                    cell = float(nums[0])
                else:
                    cell = None
            data[header[i-1]].append(cell)
#             print(cell,end=", ")
    
    #  here we want to use the information learned from the wiki links of each cities and 
    #  use it to navigate to infobox of each city
    table = infoBoxScraper(wikiurl)
    website = None
    mayor = None
    for cell in table.contents[0].contents:
        var = (re.sub('<[^<]+?>',"****",str(cell)))
        #  here we assume **** would be a unique pattern and not consist in the infobox data
        info = [x for x in (var.split('****')) if x !=""]
        if 'Website' in info:
            info.remove('Website')
            for i in info:
                m = re.findall('[\w]+\.[\w\./-]+',i)
                if len(m)>0:
                    website = m[0]
                    break
            # many time the info box stores a href link which we are unable to detect             
#             print('website found',website)
        elif 'Mayor' in info:
            info.pop(0)
            info.remove('Mayor')
            mayor = info[0]
#             print('Mayor found ',mayor )
    data['mayor'].append(mayor)
    data['website'].append(website)
            
        

New York City, Los Angeles, Chicago, Houston, Phoenix, Philadelphia, San Antonio, San Diego, Dallas, San Jose, Austin, Jacksonville, Fort Worth, Columbus, San Francisco, Charlotte, Indianapolis, Seattle, Denver, Washington, D.C., Boston, El Paso, Detroit, Nashville, Portland, Memphis, Oklahoma City, Las Vegas, Louisville, Baltimore, Milwaukee, Albuquerque, Tucson, Fresno, Mesa, Sacramento, Atlanta, Kansas City, Colorado Springs, Miami, Raleigh, Omaha, Long Beach, Virginia Beach, Oakland, Minneapolis, Tulsa, Arlington, Tampa, New Orleans, Wichita, Cleveland, Bakersfield, Aurora, Anaheim, Honolulu, Santa Ana, Riverside, Corpus Christi, Lexington, Stockton, Henderson, Saint Paul, St. Louis, Cincinnati, Pittsburgh, Greensboro, Anchorage, Plano, Lincoln, Orlando, Irvine, Newark, Toledo, Durham, Chula Vista, Fort Wayne, Jersey City, St. Petersburg, Laredo, Madison, Chandler, Buffalo, Lubbock, Scottsdale, Reno, Glendale, Gilbert, Winston–Salem, North Las Vegas, Norfolk, Chesapeake, Garland, I

Converting to csv file so that we can upload to googles bigQuerry

In [4]:
import pandas as pd
dataframe = pd.DataFrame.from_dict(data)
dataframe.to_csv('cities.csv')